In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\tg4923\Documents\GitHub\March-Madness\data\2010\Events_2010.csv')

In [3]:
df = df.loc[0:10000,:]

In [4]:
# Make a game ID to Aggregate
df['GameID'] = df['DayNum'].map(str) +"-"+ df['WTeamID'].map(str) +"-"+ df['LTeamID'].map(str)

In [5]:
df.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,GameID
0,1,2010,7,1143,1293,0,0,0,1143,600578,sub_in,7-1143-1293
1,2,2010,7,1143,1293,0,0,0,1143,600584,sub_in,7-1143-1293
2,3,2010,7,1143,1293,0,0,0,1143,600585,sub_in,7-1143-1293
3,4,2010,7,1143,1293,0,0,10,1143,600581,miss2_lay,7-1143-1293
4,5,2010,7,1143,1293,0,0,10,1143,600581,reb_off,7-1143-1293


In [6]:
df['Event_Ones'] = 1

In [13]:
team_game_stats = pd.pivot_table(df,columns=['EventType'], aggfunc = np.sum, values = 'Event_Ones',index = ['GameID','EventTeamID','WTeamID'])

In [14]:
team_game_stats.reset_index(inplace = True)

In [15]:
team_game_stats.fillna(inplace=True,value=0)

From a high level, we are looking to compare two teams at a time to determine whether or not they would win a majority of the time against each other. We want to find the most likely outcome from the tournament. 

Lets start with the assumption that we have 65 teams in the tournement. Each team has a 50% chance of winning against the other team, and there for each team has a 1 in 65 chance of winning the tournement. Obviously this is not true, the 65th team will have a much lower chance of winning the tournement over the top team. But in reality, when the top team and the 65th best team start the game, the probablility that either team wins is exaclty 50% (let's call this, oh I don't know, a "prior probability") 


For each statistical category, each team is a sample from the population that is all the teams in that category. We can keep it simple and have a normal distribution of stats accross all games in that season. (Factor in competition) 

In [16]:
team_game_stats

EventType,GameID,EventTeamID,WTeamID,assist,block,foul_pers,foul_tech,made1_free,made2_dunk,made2_jump,...,miss3_jump,reb_dead,reb_def,reb_off,steal,sub_in,sub_out,timeout,timeout_tv,turnover
0,10-1301-1209,1209,1301,13.0,1.0,23.0,0.0,6.0,0.0,6.0,...,13.0,2.0,23.0,14.0,5.0,47.0,47.0,3.0,4.0,12.0
1,10-1301-1209,1301,1301,12.0,4.0,17.0,0.0,26.0,0.0,4.0,...,13.0,3.0,27.0,12.0,8.0,30.0,31.0,2.0,3.0,9.0
2,10-1326-1241,1241,1326,6.0,1.0,14.0,0.0,8.0,0.0,5.0,...,18.0,2.0,27.0,7.0,3.0,18.0,18.0,3.0,4.0,16.0
3,10-1326-1241,1326,1326,13.0,4.0,14.0,0.0,8.0,3.0,6.0,...,13.0,1.0,31.0,12.0,11.0,15.0,15.0,0.0,3.0,9.0
4,11-1111-1423,1111,1111,11.0,7.0,24.0,1.0,33.0,0.0,11.0,...,4.0,6.0,24.0,9.0,9.0,49.0,49.0,4.0,0.0,15.0
5,11-1111-1423,1423,1111,14.0,5.0,30.0,2.0,19.0,4.0,4.0,...,13.0,2.0,16.0,15.0,5.0,47.0,47.0,4.0,8.0,15.0
6,11-1113-1442,1113,1113,17.0,4.0,10.0,0.0,23.0,2.0,5.0,...,8.0,2.0,27.0,11.0,10.0,27.0,27.0,0.0,3.0,10.0
7,11-1113-1442,1442,1113,7.0,4.0,21.0,0.0,7.0,0.0,5.0,...,7.0,2.0,14.0,14.0,5.0,22.0,22.0,1.0,5.0,18.0
8,11-1116-1108,1108,1116,5.0,3.0,30.0,0.0,24.0,0.0,9.0,...,16.0,7.0,25.0,14.0,3.0,25.0,25.0,2.0,4.0,24.0
9,11-1116-1108,1116,1116,26.0,10.0,25.0,0.0,28.0,5.0,4.0,...,14.0,6.0,31.0,14.0,14.0,32.0,32.0,1.0,4.0,9.0
